# All Domains

In [1]:
import pickle
from collections import Counter
import re
import ipy_table
import nltk
import os
import numpy as np
import spacy
import random
from spacy.pipeline import Sentencizer
import json
#from matplotlib import pyplot
import pandas as pd
import xmltodict
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

import warnings
warnings.filterwarnings('ignore')

2021-12-22 08:03:12.669131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-22 08:03:12.669180: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%run ../funcoes.py

Loading functions...


In [3]:
# spacy model
nlp = spacy.load("pt_core_news_lg")

sentencizer = nlp.create_pipe("sentencizer")
punct = [x for x in sentencizer.punct_chars]
punct.append('...')
punct.append('..')
config = {"punct_chars": punct}

#sentencizer = Sentencizer(punct_chars=punct)
nlp.add_pipe("sentencizer", first=True, config=config)

print(nlp.pipe_names)

['sentencizer', 'tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [4]:
# vargas datasets
with open('../data/aspects_camera.dat', "rb") as f: implicitCamera = pickle.load(f)
with open('../data/aspects_livro.dat', "rb") as f: implicitLivro = pickle.load(f)
with open('../data/aspects_smartphone.dat', "rb") as f: implicitSmartphone = pickle.load(f)
with open('../data/aspects_hotel.dat', "rb") as f: implicitHotel = pickle.load(f)

In [5]:
vargasDatasets= {
    'camera'     : implicitCamera,
    'livro'      : implicitLivro,
    'smartphone' : implicitSmartphone,
    'hotel'      : implicitHotel
}
datasets = [ 'camera', 'livro', 'smartphone', 'hotel' ]

In [6]:
f = open("../07_Keras/data/aspectsTest.xml","r")
xml = f.read()
aspectsTest = xmltodict.parse(xml, process_namespaces=True)

In [7]:
tag_scheme = [
'B-asp',
'I-asp'
]

In [8]:
training = {'sentences': [], 'tags' : []}
validation = {'sentences': [], 'tags' :[]}

## Cameras

In [9]:
dfCameraTest = pd.read_csv('../07_Keras/data/domain_camera_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfCameraTrain = pd.read_csv('../07_Keras/data/domain_camera_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfCameraTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1
contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfCameraTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
validation['sentences'].append(sent)
validation['tags'].append(tags)
sent = []
tags = []

cameraTest = {'sentences': [], 'tags' : []}
contSents = 0
for count, row in dfCameraTest.iterrows():
    if (row[0] != "" and count > 0):
        cameraTest['sentences'].append(sent)
        cameraTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
cameraTest['sentences'].append(sent)
cameraTest['tags'].append(tags)

## Livros

In [10]:
dfLivroTest = pd.read_csv('../07_Keras/data/domain_livro_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfLivroTrain = pd.read_csv('../07_Keras/data/domain_livro_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfLivroTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfLivroTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

livroTest = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfLivroTest.iterrows():
    if (row[0] != "" and count > 0):
        livroTest['sentences'].append(sent)
        livroTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
livroTest['sentences'].append(sent)
livroTest['tags'].append(tags)

## Smartphones

In [11]:
dfSmartphoneTest = pd.read_csv('../07_Keras/data/domain_smartphone_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfSmartphoneTrain = pd.read_csv('../07_Keras/data/domain_smartphone_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfSmartphoneTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfSmartphoneTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

smartphoneTest = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfSmartphoneTest.iterrows():
    if (row[0] != "" and count > 0):
        smartphoneTest['sentences'].append(sent)
        smartphoneTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
smartphoneTest['sentences'].append(sent)
smartphoneTest['tags'].append(tags)

### Hotel

In [12]:
dfHotelTest = pd.read_csv('../07_Keras/data/domain_hotel_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfHotelTrain = pd.read_csv('../07_Keras/data/domain_hotel_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

contSents = 0
for count, row in dfHotelTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

contTrain = round(contSents * 0.9)

sent = []
tags = []
contSents = 0
for count, row in dfHotelTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

hotelTest = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfHotelTest.iterrows():
    if (row[0] != "" and count > 0):
        hotelTest['sentences'].append(sent)
        hotelTest['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
hotelTest['sentences'].append(sent)
hotelTest['tags'].append(tags)

In [13]:
print(len(training['sentences']))
print(len(validation['sentences']))

print(len(cameraTest['sentences']))
print(len(livroTest['sentences']))
print(len(smartphoneTest['sentences']))

1230
136
81
88
106


## Bert Multilingual

In [14]:
transformer = 'bert-base-multilingual-uncased'

In [15]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [16]:
from NERDA.models import NERDA
model = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
%%time
model.train()


 Epoch 1 / 40


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:34<00:00,  2.02s/it]


Train Loss = 0.4432393990849194 Valid Loss = 0.21418272353270473

 Epoch 2 / 40


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:32<00:00,  1.93s/it]


Train Loss = 0.20866538874412838 Valid Loss = 0.19055270447450526

 Epoch 3 / 40


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:17<00:00,  1.02s/it]


Train Loss = 0.0002420224409051068 Valid Loss = 0.42158754695864287

 Epoch 40 / 40


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:16<00:00,  1.00it/s]

Train Loss = 0.000328079792521557 Valid Loss = 0.4218040836208007
CPU times: user 2d 19h 30min 7s, sys: 32min 27s, total: 2d 20h 2min 35s
Wall time: 10h 54min 56s


'Model trained successfully'

In [18]:
def result2aspecList(y_pred, X_test):
    aspectsList = []
    aspect = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for sent, sent_pred in zip(X_test, y_pred):
        #print(len(sent),len(sent_pred))
        for token, tag in zip(sent, sent_pred):
            if tag != 'O':
                #print(tag, token)
                #print(tag, token['word.lower()'])
                if tag == 'B-asp' and aspect:
                    auxAspect = " ".join(aspect).lower()
                    if (auxAspect not in stopwords and len(auxAspect) > 1):
                        aspectsList.append(auxAspect)
                    aspect = []
                if token not in [',',':']:
                    aspect.append(token.lower())

    aspectsList = list(set(aspectsList))
    aspectsList.sort()    
    return aspectsList

### Cameras

In [19]:
# carregando tipos
domain = "camera"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [20]:
model.evaluate_performance(cameraTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.385417,0.359223,0.415730
1,I-asp,0.117647,0.153846,0.095238
0,AVG_MICRO,0.315385,NaN,NaN
0,AVG_MICRO,0.251532,NaN,NaN


In [21]:
y_pred = model.predict(cameraTest['sentences'])

In [22]:
aspectsCamera = result2aspecList(y_pred, cameraTest['sentences'])
resCamera = calculaMetricas(
    aspectsCamera, 
    aspectsTest['datasets']['camera'], 
    tipos,
    lemma=False, 
    output=False)
resCamera

{'totalAspects': 104,
 'pred': 58,
 'tp': 50,
 'fp': 36,
 'fn': 54,
 'precisao': 0.5813953488372093,
 'precisaoLaplace': 0.5795454545454546,
 'revocacao': 0.4807692307692308,
 'medidaf': 0.5263157894736843,
 'totalExplicitos': 91,
 'explicitosOk': 21,
 'pExplicitos': 0.23076923076923078,
 'explicitsList': ['recursos',
  'produto',
  'resolução',
  'visor',
  'funções',
  'fotografia',
  'tela',
  'câmera',
  'praticidade',
  'preço',
  'investimento',
  'menu',
  'zoom',
  'imagens',
  'bateria',
  'flash',
  'máquina',
  'cartão de memória',
  'fotos',
  'manuseio',
  'foco'],
 'totalImplicitos': 13,
 'implicitosOk': 1,
 'pImplicitos': 0.07692307692307693,
 'implicitsList': ['compacta'],
 'tipos': {'Qualification': {'Adjective': 4}}}

In [23]:
xml = dicttoxml(resCamera, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    

## Livros

In [24]:
# carregando tipos
domain = "livro"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [25]:
model.evaluate_performance(livroTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.783133,0.8125,0.755814
1,I-asp,0.153846,0.5000,0.090909
0,AVG_MICRO,0.737430,NaN,NaN
0,AVG_MICRO,0.468489,NaN,NaN


In [26]:
y_pred = model.predict(livroTest['sentences'])

In [27]:
aspectsLivro = result2aspecList(y_pred, livroTest['sentences'])
resLivro = calculaMetricas(
    aspectsLivro, 
    aspectsTest['datasets'][domain], 
    tipos,
    output=False)
resLivro

{'totalAspects': 87,
 'pred': 31,
 'tp': 58,
 'fp': 9,
 'fn': 29,
 'precisao': 0.8656716417910447,
 'precisaoLaplace': 0.855072463768116,
 'revocacao': 0.6666666666666666,
 'medidaf': 0.7532467532467532,
 'totalExplicitos': 82,
 'explicitosOk': 22,
 'pExplicitos': 0.2682926829268293,
 'explicitsList': ['estilo',
  'livro',
  'personagem',
  'história',
  'palavra',
  'começo',
  'trama',
  'historia',
  'leitura',
  'obra',
  'final',
  'romance',
  'situaçoês',
  'escritoras',
  'crítica',
  'escritor',
  'protagonista',
  'fim',
  'narrativa',
  'narrador',
  'personagens',
  'início'],
 'totalImplicitos': 5,
 'implicitosOk': 0,
 'pImplicitos': 0.0,
 'implicitsList': [],
 'tipos': {}}

In [32]:
xml = dicttoxml(resLivro, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())

## Smartphones

In [29]:
# carregando tipos
domain = "smartphone"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [30]:
model.evaluate_performance(smartphoneTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.460227,0.460227,0.460227
1,I-asp,0.355556,0.400000,0.320000
0,AVG_MICRO,0.438914,NaN,NaN
0,AVG_MICRO,0.407891,NaN,NaN


In [31]:
y_pred = model.predict(smartphoneTest['sentences'])

In [35]:
aspectsSmartphone = result2aspecList(y_pred, smartphoneTest['sentences'])
resSmartphone = calculaMetricas(
    aspectsSmartphone, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, output=False)
resSmartphone

{'totalAspects': 158,
 'pred': 78,
 'tp': 76,
 'fp': 41,
 'fn': 82,
 'precisao': 0.6495726495726496,
 'precisaoLaplace': 0.6470588235294118,
 'revocacao': 0.4810126582278481,
 'medidaf': 0.5527272727272727,
 'totalExplicitos': 134,
 'explicitosOk': 33,
 'pExplicitos': 0.2462686567164179,
 'explicitsList': ['produto',
  'funcionalidades',
  'gps',
  'câmera',
  'touch',
  'preço',
  'sd de memória',
  'fabricante',
  'funções',
  'aparelho',
  'wi-fi',
  'designer',
  'visor',
  'fotografia',
  'tamanho',
  'bateria',
  'custo-benefício',
  'celular',
  'rádio',
  'display',
  'internet',
  'smartphone',
  'camera',
  'volume de áudio',
  'memoria',
  'recursos',
  'design',
  'memória interna',
  'camera digital',
  'nokia',
  'marca',
  'luz de flash',
  'fotos'],
 'totalImplicitos': 24,
 'implicitosOk': 4,
 'pImplicitos': 0.16666666666666666,
 'implicitsList': ['congela', 'trava', 'prático', 'moderno'],
 'tipos': {'Event': {'Verb': 3}, 'Qualification': {'Adjective': 2}}}

In [36]:
xml = dicttoxml(resSmartphone, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())

## Hotel

In [37]:
# carregando tipos
domain = "hotel"
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [38]:
model.evaluate_performance(hotelTest)

,Level,F1-Score,Precision,Recall
0,B-asp,0.765847,0.791444,0.741855
1,I-asp,0.380282,0.529412,0.296703
0,AVG_MICRO,0.706011,NaN,NaN
0,AVG_MICRO,0.573065,NaN,NaN


In [39]:
y_pred = model.predict(hotelTest['sentences'])

In [40]:
aspectsHotel = result2aspecList(y_pred, hotelTest['sentences'])
resHotel = calculaMetricas(
    aspectsHotel, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, output=False)
resHotel

{'totalAspects': 518,
 'pred': 126,
 'tp': 352,
 'fp': 48,
 'fn': 166,
 'precisao': 0.88,
 'precisaoLaplace': 0.8781094527363185,
 'revocacao': 0.6795366795366795,
 'medidaf': 0.7668845315904139,
 'totalExplicitos': 292,
 'explicitosOk': 35,
 'pExplicitos': 0.11986301369863013,
 'explicitsList': ['telefone',
  'preço',
  'porteiro',
  'estacionamento',
  'frigobar',
  'garagem',
  'rodoviária',
  'serviço',
  'estabelecimento',
  'lojas',
  'lavanderia',
  'pia',
  'shopping',
  'recepção',
  'piscina',
  'conforto',
  'elevador',
  'aparência',
  'aeroporto',
  'apartamento',
  'rua',
  'chuveiro',
  'gerência',
  'portaria',
  'limpeza',
  'quarto',
  'funcionários',
  'cozinha',
  'serviço de quarto',
  'cidade',
  'hotel',
  'carpete',
  'atendimento',
  'instalações',
  'localização'],
 'totalImplicitos': 226,
 'implicitosOk': 55,
 'pImplicitos': 0.24336283185840707,
 'implicitsList': ['próximo',
  'barato',
  'custo',
  'sujo',
  'telefone',
  'cortina',
  'estacionamento',
  'li

In [41]:
xml = dicttoxml(resHotel, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_alldomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())